In [1]:
import numpy as np
import qililab as ql

from utils.qst_qpt_helper_functions import *

from qiboconnection.api import API
from qiboconnection.connection import ConnectionConfiguration

import os

import matplotlib.pyplot as plt

from scipy.optimize import curve_fit


api = API(ConnectionConfiguration(username="qat", api_key="meow"))


api.select_device_id(9)

2024-02-18 18:00:54,982 - qm - INFO     - Starting session: e484d76d-0173-4bb6-9bb6-124400ff6d8c


Is setting the logger perhaps a bit ugly for remote users to have to do manually? I have not seen any experiment where it is preferred to not be set to a higher level

In [2]:
ql.logger.setLevel(40)  # Set qililab's logger to a higher level so it only shows error messages

In [3]:
# Amplitude set in Qblox terms (0.5 times mV)
Time_START = 0
Time_STOP = 200000
Time_STEP = 20  ## this had two more zeros, but with the software loop if was taking 7more than 7 minutes to send.

SQUARE_duration = 4000
GAUSS_duration = 2000

NSHOTS = 2000

In [36]:
# The shape of the pulses is introduced here
gauss_wf = ql.Gaussian(amplitude=1, duration=GAUSS_duration, num_sigmas=4.0)  # Drag pulse executing pi rotation
square_wf = ql.Square(amplitude=1, duration=SQUARE_duration)  # square wavefront (readout pulse)

weights_shape = ql.Square(amplitude=1, duration=SQUARE_duration)  # This shape implies that the weights are uniform
## this uniform weighting has nothing to do with the thresholding, it's because Qblox allows for weighting the points
## of the waveform that you sent differently. This feature in particular is designed for aiding in making the square
## pulses as square as possible (since one has to correct the edges for a digital square signal). We set it to be uniform
## always, but I believe we are making this correction elsewhere. Check with Guille though.

In [5]:
# Building the Qprogram
qp_t1 = ql.QProgram()

# Defining the variable Frequency for the driving pulse
wait_time = qp_t1.variable(ql.Domain.Time)  # Possible domains: Scalar, Time, Frequency, Phase and Voltage

# Averaging over NSHOTS iterations
with qp_t1.average(NSHOTS):
    # Loop over all wait times
    with qp_t1.for_loop(variable=wait_time, start=Time_START, stop=Time_STOP, step=Time_STEP):
        qp_t1.play(bus="drive_q0_bus", waveform=gauss_wf)  # execute the pi pulse
        qp_t1.wait(bus="drive_q0_bus", duration=wait_time)  # wait

        qp_t1.sync()  # so that the next bus (for the readout) executes after the previous (drive)
        ## a sync must be added between different buses if we want them to be consecutive. Each bus
        ## refers to either a particular qubit's sequence of DRAGs, a particular qubit's readout pulse
        ## or a particular qubit pair's sequence of two-qubit gates

        # READOUT PULSE
        qp_t1.play(bus="readout_q0_bus", waveform=square_wf, wait_time=4)  ## execute readout pulse
        qp_t1.acquire(
            bus="readout_q0_bus", weights=ql.IQPair(I=weights_shape, Q=weights_shape)
        )  # Collect data from readout

full_qprogram = qp_t1.to_dict()

# This code can't work yet because the sync after a wait time is not properly implemented yet.

In [8]:
Wait_duration_list = np.linspace(Time_START, Time_STOP, Time_STEP)

In [40]:
result_id = api.execute(qprogram=full_qprogram)[0]
print(result_id)

8611


### Post-processing of data is now going to the same function as if it were coming from a circuit, but the post-processing will likely have to be different.

In [ ]:
data_probabilities = process_returned_dataformat(result_id, nqubits=1)

In [ ]:
probabilities_excited = data_probabilities[:, 1]

fit_up2 = -1
coeffs_fit = np.polyfit(Wait_duration_list[:fit_up2], np.log10(probabilities_excited[:fit_up2]), 1)
poly1d_fn = np.poly1d(coeffs_fit)  # poly1d_fn is now a function which takes in x and returns an estimate for y
exponent = coeffs_fit[0]
T1 = -1 / exponent

# Plot the data and the fitted function
plt.figure(figsize=(8, 6))
plt.plot(Wait_duration_list, np.log10(probabilities_excited), label="data")
plt.plot(Wait_duration_list, poly1d_fn(Wait_duration_list), "r", label=f"Fitted T1={T1/1000:.3f} us")
plt.legend()
plt.xlabel("Wait time (ns)")
plt.ylabel("Prob($|1\\rangle$)")